In [11]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:

# Configurar el límite de caracteres a mostrar
pd.set_option('display.max_colwidth', None)

# Tu código de extracción y creación del DataFrame aquí


In [4]:
driver = webdriver.Chrome('str/chromedriver')

/tmp/ipykernel_51338/3008020181.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('str/chromedriver')


In [5]:
url = 'https://www.data.jma.go.jp/multi/quake/index.html?lang=es'
driver.get(url)

In [9]:
wait = WebDriverWait(driver,10)

In [12]:
tabla = wait.until(EC.presence_of_element_located((By.ID, 'quakeindex_table')))

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=113.0.5672.92)
Stacktrace:
#0 0x562a8cbe3133 <unknown>
#1 0x562a8c917966 <unknown>
#2 0x562a8c8f1cec <unknown>
#3 0x562a8c978c0f <unknown>
#4 0x562a8c98bc2b <unknown>
#5 0x562a8c9739a3 <unknown>
#6 0x562a8c94846a <unknown>
#7 0x562a8c94955e <unknown>
#8 0x562a8cba2cae <unknown>
#9 0x562a8cba68fe <unknown>
#10 0x562a8cbaff20 <unknown>
#11 0x562a8cba7923 <unknown>
#12 0x562a8cb7ac0e <unknown>
#13 0x562a8cbcab08 <unknown>
#14 0x562a8cbcac97 <unknown>
#15 0x562a8cbdb113 <unknown>
#16 0x7f3ad2694b43 <unknown>


In [123]:
time.sleep(2) 

In [124]:
filas = tabla.find_elements(By.XPATH, './/tr')
# Recorrer las filas de la tabla y extraer los enlaces
enlaces = []
for i in range(1, len(filas)):  # Ignorar la primera fila (encabezados de columna)
    fila = filas[i]  # Obtener la fila actual
    
    # Obtener el enlace
    enlace = fila.find_element(By.XPATH, './/td[1]/a').get_attribute('href')
    enlaces.append(enlace)
    

# Guardar los enlaces en un DataFrame
df_enlaces = pd.DataFrame(enlaces, columns=['Enlace'])



                                                                                      Enlace
0    https://www.data.jma.go.jp/multi/quake/quake_detail.html?eventID=20230517143244&lang=es
1    https://www.data.jma.go.jp/multi/quake/quake_detail.html?eventID=20230517142801&lang=es
2    https://www.data.jma.go.jp/multi/quake/quake_detail.html?eventID=20230517112947&lang=es
3    https://www.data.jma.go.jp/multi/quake/quake_detail.html?eventID=20230517003740&lang=es
4    https://www.data.jma.go.jp/multi/quake/quake_detail.html?eventID=20230516210104&lang=es
..                                                                                       ...
266  https://www.data.jma.go.jp/multi/quake/quake_detail.html?eventID=20230420094616&lang=es
267  https://www.data.jma.go.jp/multi/quake/quake_detail.html?eventID=20230420022632&lang=es
268  https://www.data.jma.go.jp/multi/quake/quake_detail.html?eventID=20230419094044&lang=es
269  https://www.data.jma.go.jp/multi/quake/quake_detail.html?eventID=

In [163]:
df_enlaces.to_csv('str/historiallink.csv',index=False)  

In [162]:
def scrape_page(url):
    # Configurar Selenium
    options = Options()
    options.add_argument("--headless")  # Ejecutar en modo headless (sin ventana)
    service = Service('str/chromedriver')  # Reemplaza con la ubicación de tu archivo chromedriver

    # Inicializar el controlador de Selenium
    driver = webdriver.Chrome(service=service, options=options)
    driver.get(url)
    
    # Esperar 2 segundos para que se cargue la página
    time.sleep(2)
    
    # Buscar la tabla por su ID
    table = driver.find_element(By.ID, 'quakeindex_table')
    
    # Extraer los datos de la tabla
    data = []
    rows = table.find_elements(By.TAG_NAME, 'tr')
    
    for row in rows[1:]:  # Omitir la primera fila (encabezados)
        columns = row.find_elements(By.TAG_NAME, 'td')
        fecha = columns[0].text
        longitud = columns[1].text
        latitud = columns[2].text
        magnitud = columns[3].text
        profundidad = columns[4].text
        lugar = columns[5].text
        
        data.append({
            'Fecha detectado el terremoto': fecha,
            'longitud': longitud,
            'latitud': latitud,
            'Magnitud': magnitud,
            'profundidad': profundidad,
            'lugar': lugar
        })
    
    # Cerrar el controlador de Selenium
    driver.quit()
    
    return pd.DataFrame(data)

# Crear una lista para almacenar los DataFrames raspados
dfs = []

# Aplicar la función de scraping a cada enlace en el DataFrame
for index, row in df_enlaces.iterrows():
    url = row['Enlace']
    scraped_data = scrape_page(url)
    
    if scraped_data is not None:
        dfs.append(scraped_data)

# Concatenar los DataFrames en uno nuevo
new_df = pd.concat(dfs, ignore_index=True)

# Imprimir el nuevo DataFrame resultante
print(new_df)

    Fecha detectado el terremoto longitud latitud Magnitud    profundidad   
0               2023/05/17 14:29    29.9N  130.1E      2.7           10km  \
1               2023/05/17 14:24    24.8N  125.2E      3.7           20km   
2               2023/05/17 11:26    29.9N  130.0E      3.1           10km   
3               2023/05/17 00:34    29.9N  130.0E      3.0           10km   
4               2023/05/16 20:56    33.4N  139.3E      4.7           10km   
..                           ...      ...     ...      ...            ...   
264             2023/04/20 09:42    26.9N  141.5E      4.4  Poco profundo   
265             2023/04/20 02:23    34.0N  135.1E      2.8           10km   
266             2023/04/19 09:37    37.5N  137.2E      3.2           10km   
267             2023/04/18 21:56    29.8N  129.9E      2.0           10km   
268             2023/04/18 17:03    26.7N  127.8E      3.2           50km   

                                           lugar  
0                 Mares 

In [176]:
new_df.to_csv('str/data_actualizadajapon.csv')